# Google Drive Access

In [17]:
# Configure access to Google Colab folder to save results later
# Reference: https://hackmd.io/@BrownDeepLearningS24/BkIT3sW6s

import os
import sys

results_path = ''

isColab = "google.colab" in sys.modules
if isColab:
  from google.colab import drive
  results_path = ("/content/drive" + "/My Drive/Colab Notebooks/")
  sys.path.append(results_path)

## Preprocessing

In [18]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [20]:
# Determine the number of unique values in each column.
unique_values_count_per_column = application_df.nunique(axis=0)
print(unique_values_count_per_column)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [21]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_value_counts = application_df.groupby(['APPLICATION_TYPE']).size()
print(application_type_value_counts.sort_values(ascending=False))

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
dtype: int64


In [22]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Chosen cutoff value is 500:
application_types_to_replace = application_type_value_counts[application_type_value_counts < 500].index.to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
revised_application_type_value_counts = application_df['APPLICATION_TYPE'].value_counts()
print(revised_application_type_value_counts.sort_values(ascending=False))

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [23]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_type_value_counts = application_df.groupby(['CLASSIFICATION']).size()
print(classification_type_value_counts.sort_values(ascending=False))

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C2380        1
C2500        1
C2561        1
C8210        1
Length: 71, dtype: int64


In [24]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts = application_df.groupby(['CLASSIFICATION']).size()
classification_value_counts_gt_1 = classification_value_counts[classification_value_counts > 1]
print(classification_value_counts_gt_1.sort_values(ascending=False))

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C1256        2
C1246        2
C3200        2
C1267        2
C1234        2
dtype: int64


In [25]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Chosen cutoff value is 1000:
classifications_to_replace = classification_value_counts[classification_value_counts < 1000].index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
revised_classification_value_counts = application_df['CLASSIFICATION'].value_counts()
print(revised_classification_value_counts.sort_values(ascending=False))

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [26]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']

encoded_application_df = pd.get_dummies(data=application_df, columns=categorical_columns, dtype=int)


In [27]:
# Split our preprocessed data into our features and target arrays
y = encoded_application_df['IS_SUCCESSFUL']              # target
X = encoded_application_df.drop(columns='IS_SUCCESSFUL') # features

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y)

In [28]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
# Inspect inputs to check the number of feature inputs (columns)
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


## Compile, Train and Evaluate the Model

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
# Define 80 neurons in the first layer.
# Allow for the 43 input features (number of columns in 'X') plus 1 bias input each, so 43+1=44 inputs here.
# Expected number of parameters for this layer = 80*(43+1) = 3,520
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=43))

# Second hidden layer
# Define 30 neurons in the second layer (about 2/3 the number of input features, rounded up to nearest round number).
# (For this and other guidelines, refer to https://medium.com/geekculture/introduction-to-neural-network-2f8b8221fbd3)
# Allow for the 80 outputs from the first layer as inputs here, plus 1 bias input each, so 80+1=81 inputs here.
# Expected number of parameters for this layer = 30*(80+1) = 2,430
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
# Define 1 neuron in the output layer (to determine a binary output i.e. Successful/Unsuccessful)
# Expected number of parameters for this layer = number of neurons in previous layer, plus 1 bias input, so 30+1=31.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 80)                  │           3,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Define folder path to save intermediate/final model weights
checkpoint_filepath = results_path + 'checkpoint.weights.h5'

# Define a callback function we can use to save the model weights every 5 epochs
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, save_freq=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, callbacks=[checkpoint], epochs=100, verbose=2)

Epoch 1/100
804/804 - 5s - 7ms/step - accuracy: 0.7177 - loss: 0.5742
Epoch 2/100
804/804 - 5s - 6ms/step - accuracy: 0.7305 - loss: 0.5547
Epoch 3/100
804/804 - 4s - 5ms/step - accuracy: 0.7315 - loss: 0.5518
Epoch 4/100
804/804 - 4s - 5ms/step - accuracy: 0.7299 - loss: 0.5505
Epoch 5/100
804/804 - 5s - 7ms/step - accuracy: 0.7324 - loss: 0.5487
Epoch 6/100
804/804 - 4s - 5ms/step - accuracy: 0.7313 - loss: 0.5477
Epoch 7/100
804/804 - 6s - 8ms/step - accuracy: 0.7326 - loss: 0.5473
Epoch 8/100
804/804 - 4s - 5ms/step - accuracy: 0.7338 - loss: 0.5463
Epoch 9/100
804/804 - 5s - 6ms/step - accuracy: 0.7339 - loss: 0.5459
Epoch 10/100
804/804 - 5s - 6ms/step - accuracy: 0.7339 - loss: 0.5447
Epoch 11/100
804/804 - 4s - 5ms/step - accuracy: 0.7329 - loss: 0.5450
Epoch 12/100
804/804 - 4s - 5ms/step - accuracy: 0.7332 - loss: 0.5442
Epoch 13/100
804/804 - 6s - 8ms/step - accuracy: 0.7357 - loss: 0.5443
Epoch 14/100
804/804 - 4s - 5ms/step - accuracy: 0.7353 - loss: 0.5438
Epoch 15/100
80

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7286 - loss: 0.5648
Loss: 0.5648255348205566, Accuracy: 0.7286297082901001


In [34]:
# Export our model to HDF5 file
nn.save(results_path + 'AlphabetSoupCharity.h5')